# Análise Ranking CCONF

Neste notebook vamos analisar a aba municípios da planilha que contém os dados sobre a aplicação dos critérios elaborados pela CCONF. Vamos lá!

Começamos importando os dados da planilha para dentro de um Data Frame para podermos manipulá-los no python:

Vamos criar duas variáveis que guardam as colunas que: 
a) São relacionadas à capag:
b) Não são relacionadas à capag

In [7]:
col_capag = "S:W, Y:AA, AL:AN, AS, AU: BA, BD:BF, BM, BO, BQ, BS, BU"
col_nao_capag = "B:R,X,AB:AK, AO:AR, AT, BB:BC, BG:BL, BN,BP,BR,BT"
col_gerais = "A,BV, BW,BX,BY,BZ"

Criando os Data Frames usando a library pandas

In [8]:
import pandas as pd
import numpy as np

data_capag = pd.read_excel(r'C:\Users\fioro\PycharmProjects\ranking_cconf\assets\Testes_Ranking_CCONF_2020-CAPAG.xlsx', 'Municípios', usecols = col_gerais+col_capag)
print(data_capag.head(5))

data_nao_capag = pd.read_excel(r'C:\Users\fioro\PycharmProjects\ranking_cconf\assets\Testes_Ranking_CCONF_2020-CAPAG.xlsx', 'Municípios', usecols = col_gerais+col_nao_capag)
print(data_nao_capag.head(5))

      ENTE  D2_00010  D2_00011  D2_00012  D2_00023  D2_00024  D2_00028  \
0  4200556       1.0       1.0       1.0       1.0       1.0       1.0   
1  4317004       1.0       1.0       1.0       1.0       1.0       1.0   
2  3165578       1.0       1.0       1.0       1.0       1.0       1.0   
3  4320321       1.0       1.0       1.0       1.0       1.0       1.0   
4  2103554       1.0       1.0       1.0       1.0       1.0       1.0   

   D2_00033  D3_00002  D3_00003  ...  D4_00001  D4_00008  D4_00010  D4_00012  \
0       1.0       1.0       1.0  ...       1.0       1.0       1.0       1.0   
1       1.0       1.0       1.0  ...       1.0       1.0       1.0       1.0   
2       1.0       1.0       1.0  ...       1.0       1.0       1.0       1.0   
3       1.0       1.0       1.0  ...       1.0       1.0       1.0       1.0   
4       1.0       1.0       1.0  ...       1.0       1.0       1.0       1.0   

   D4_00014  D4_00016                        Ente  Total  Capital?   Id  


Os nomes dos municípios e e estados estão agrupados na coluna "BV", vamos separá-los

In [9]:
#data['Ente'] = Cod_UF.ESTADO.str[-2:]
